In [1]:
"""
# 전처리 배분

1(성준) - 0~9,   40~49, 80~89, 120, 
       - credit_card_balance.csv
       - POS_CASH_balance.csv
       - previous_application.csv
       - installments_payments.csv
2(장호) - 10~19, 50~59, 90~99, 121
3(두영) - 20~29, 60~69, 100~109, 122, 
       - bureau.csv,
4(혜빈) - 30~39, 70~79, 110~119, 123, 
       - bureau_balance.csv
5(준혁) -  
"""

"""
# 데이터 목록
application_test.csv
application_train.csv
credit_card_balance.csv
POS_CASH_balance.csv
previous_application.csv
installments_payments.csv
bureau.csv
bureau_balance.csv
sample_submission.csv
"""

'\n# 데이터 목록\napplication_test.csv\napplication_train.csv\nPOS_CASH_balance.csv\nbureau.csv\nbureau_balance.csv\ncredit_card_balance.csv\ninstallments_payments.csv\nprevious_application.csv\nsample_submission.csv\n'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
import user_functions as uf

import platform
import matplotlib
%matplotlib inline
# 한글 출력되게 설정
uf.korean()

## installments_payments.csv
---

In [109]:
inst_paym = pd.read_csv(uf.get_f_path()+"installments_payments.csv")

In [110]:
inst_paym.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   SK_ID_CURR              int64  
 2   NUM_INSTALMENT_VERSION  float64
 3   NUM_INSTALMENT_NUMBER   int64  
 4   DAYS_INSTALMENT         float64
 5   DAYS_ENTRY_PAYMENT      float64
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


In [141]:
# "AMT_PAYMENT" 실제 상환액
# 결측값은 상환하지 않았다고 판단하여 0으로 값을 대치함
# uf.check_feature(inst_paym, "AMT_PAYMENT")
inst_paym.loc[:, "AMT_PAYMENT"].fillna(0, inplace=True)

# "AMT_INSTALMENT" 할부금 총액
# "AMT_INSTALMENT" 할부금 총액 0인 행 제외
# uf.check_feature(inst_paym, "AMT_INSTALMENT")
drop_index = inst_paym.loc[inst_paym["AMT_INSTALMENT"] == 0].index
inst_paym.drop(drop_index, inplace=True)

In [152]:
# "AMT_PAYMENT" 실제 상환액 / "AMT_INSTALMENT" 할부금 총액
temp_payment = inst_paym[["SK_ID_CURR", "AMT_PAYMENT"]].groupby("SK_ID_CURR").mean()
temp_install = inst_paym[["SK_ID_CURR", "AMT_INSTALMENT"]].groupby("SK_ID_CURR").mean()
temp_merge = pd.merge(temp_payment, temp_install, left_on="SK_ID_CURR", right_on="SK_ID_CURR")
temp_merge["과거_상환액/총액"] = temp_merge["AMT_PAYMENT"] / temp_merge["AMT_INSTALMENT"]

# new_inst_paym Dataframe 만들기
new_inst_paym = temp_merge[["과거_상환액/총액"]]

# 저장
new_inst_paym.to_csv("../pre_data/new_installments_payments.csv")